In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import os
import matplotlib.pyplot as plt
import mysql.connector
from datetime import datetime, timedelta

# Customer Database Dataset

In [ ]:
mydb = mysql.connector.connect(
  host="host",
  user="user",
  password="pass",
  database="database",
  port = 2701
)

In [ ]:
mycursor = mydb.cursor()

In [ ]:
mycursor.execute(
"""
query
"""
)

In [ ]:
result = mycursor.fetchall()
# result

In [ ]:
customer_dataset = pd.DataFrame(result)
customer_dataset

# Return Customer Dataset

In [ ]:
directory = r'dir'
all_files = glob.glob(os.path.join(directory, "*.csv"))

In [ ]:
dataframes = []

column_return_code = ["return_code"]

for file in all_files:
    df = pd.read_csv(file)

    for col in column_awb:
        if col in df.columns:
            df = df.rename(columns={col: 'return_code'})
            break

    dataframes.append(df)

return_dataset = pd.concat(dataframes, ignore_index=True)
return_dataset

In [ ]:
# missing_awb = return_dataset['return_code'].isna().sum()
# missing_awb

# ETA Dataset

In [ ]:
eta_dataset = pd.read_csv(r'dir')
eta_dataset['Region'] = eta_dataset['Region'].str.lower()
eta_dataset

# Pre-Processing

## Remove Return Data

In [ ]:
customer_dataset['0'] = customer_dataset[5].astype(str)
return_dataset['return_code'] = return_dataset['return_code'].astype(str)

In [ ]:
dataset = customer_dataset[~customer_dataset[5].isin(return_dataset['return_code'])]
dataset

## Remove Inaccurate Date

In [ ]:
dataset = dataset[dataset.iloc[:, 3] != '0000-00-00']
dataset

## Replace and Lower Location Name

In [ ]:
dataset.iloc[:, 6] = dataset.iloc[:, 6].str.replace('Kab. ', '').str.replace('Kota ', '').str.lower()
dataset

In [ ]:
# missing_lsc = dataset[4].isna().sum()
# missing_lsc

# Replace Missing LSC

In [ ]:
def ambil_eta(region):
    matched_row = eta_dataset[eta_dataset['Region'] == region]

    if not matched_row.empty:
        return matched_row.iloc[0]['Est. Time Arrival']
    else:
        return '5'

dataset['Est. Time Arrival'] = dataset.iloc[:, 6].apply(ambil_eta)
dataset

In [ ]:
def tambah_hari(tanggal, tambahan_hari):
    if pd.notnull(tanggal):
        try:
            if isinstance(tanggal, datetime):
                tanggal = tanggal.strftime('%Y-%m-%d')
            tanggal = datetime.strptime(str(tanggal), '%Y-%m-%d')
            tambahan_hari = int(tambahan_hari)
            return (tanggal + timedelta(days=tambahan_hari)).strftime('%Y-%m-%d')
        except ValueError:
            return 'Invalid'
    else:
        return 'Invalid'

def isi_kolom_index_4(tanggal, est_time_arrival):
    if pd.isnull(tanggal):
        return tambah_hari(tanggal, est_time_arrival)
    else:
        return tambah_hari(tanggal, est_time_arrival)

dataset[4] = dataset.apply(lambda row: isi_kolom_index_4(row[3], row['Est. Time Arrival']), axis=1)
dataset

In [ ]:
# print(dataset.dtypes)

# Churn

In [ ]:
dataset['Est. Time Arrival'] = pd.to_numeric(dataset['Est. Time Arrival'], errors='coerce')

In [ ]:
mapping_masa_habis = {
    "product": 00
}

def get_masa_habis(nama_produk):
    return mapping_masa_habis.get(nama_produk, None)

dataset['Masa Habis'] = dataset[1].apply(get_masa_habis)
dataset['Estimasi Habis'] = dataset[2] * dataset['Masa Habis']
dataset

In [ ]:
dataset['Masa Habis'] = pd.to_numeric(dataset['Masa Habis'], errors='coerce').fillna(0).astype(int)
dataset['Estimasi Habis'] = pd.to_numeric(dataset['Estimasi Habis'], errors='coerce').fillna(0).astype(int)

In [ ]:
dataset

In [ ]:
print('Tipe Data')
print(dataset.dtypes)
print('---------------------------------')

missing_values = dataset.isna().sum()
print('Missing Value')
print(missing_values)

In [ ]:
mean_estimasi_habis = dataset['Estimasi Habis'].mean().astype(int)
median_estimasi_habis = dataset['Estimasi Habis'].median()
mode_estimasi_habis = dataset['Estimasi Habis'].mode()
min_estimasi_habis = dataset['Estimasi Habis'].min()
max_estimasi_habis = dataset['Estimasi Habis'].max()

print(f"Mean 'Estimasi Habis': {mean_estimasi_habis}")
print(f"Median 'Estimasi Habis': {median_estimasi_habis}")
print(f"Mode 'Estimasi Habis': {mode_estimasi_habis[0] if not mode_estimasi_habis.empty else 'No mode'}")
print(f"Min 'Estimasi Habis': {min_estimasi_habis}")
print(f"Max 'Estimasi Habis': {max_estimasi_habis}")

In [ ]:
def tambah_hari(tanggal, tambahan_hari):
    if pd.notnull(tanggal):
        try:
            if isinstance(tanggal, datetime):
                tanggal = tanggal.strftime('%Y-%m-%d')
            tanggal = datetime.strptime(str(tanggal), '%Y-%m-%d')
            tambahan_hari = int(tambahan_hari)
            return (tanggal + timedelta(days=tambahan_hari)).strftime('%Y-%m-%d')
        except ValueError:
            return 'Invalid'
    else:
        return 'Invalid'

def isi_kolom_tanggal_habis(tanggal, estimasi_habis):
    if pd.isnull(tanggal):
        return tambah_hari(tanggal, estimasi_habis)
    else:
        return tambah_hari(tanggal, estimasi_habis)

dataset['Tanggal Habis'] = dataset.apply(lambda row: isi_kolom_tanggal_habis(row[4], row['Estimasi Habis']), axis=1)

dataset

In [ ]:
tanggal_sekarang = datetime.now()
# dataset['Interval'] = (tanggal_sekarang - pd.to_datetime(dataset['Tanggal Habis'], errors='coerce')).dt.days
dataset['Interval'] = (tanggal_sekarang - pd.to_datetime(dataset[4], errors='coerce')).dt.days
dataset.loc[dataset['Tanggal Habis'] == 'Invalid', 'Interval'] = 'Invalid'
dataset

In [ ]:
dataset['Interval'] = pd.to_numeric(dataset['Interval'], errors='coerce')

In [ ]:
dataset

In [ ]:
mean_interval = dataset['Interval'].mean().astype(int)
median_interval = dataset['Interval'].median()
mode_interval = dataset['Interval'].mode()
min_interval = dataset['Interval'].min()
max_interval = dataset['Interval'].max()

print(f"Mean 'Interval': {mean_interval}")
print(f"Median 'Interval': {median_interval}")
print(f"Mode 'Interval': {mode_interval[0] if not mode_interval.empty else 'No mode'}")
print(f"Min 'Interval': {min_interval}")
print(f"Max 'Interval': {max_interval}")

In [ ]:
def determine_churn(row):
    if pd.isnull(row['Interval']):
        return 'Invalid'
    elif row['Interval'] < 0:
        return 'Tidak'
    elif row['Estimasi Habis'] >= row['Interval'] :
        return 'Tidak'
    else:
        return 'Ya'

dataset['Churn'] = dataset.apply(determine_churn, axis=1)

dataset

In [ ]:
churn_counts = dataset['Churn'].value_counts()
churn_counts

In [ ]:
churn_counts_per_product = dataset.groupby(1)['Churn'].value_counts().unstack(fill_value=0)
churn_counts_per_product

# Analysis

In [ ]:
brand_officer = ['product']
dataset_filtered = dataset[dataset[1].isin(brand_officer)]

churn_percentage = dataset_filtered.groupby(1)['Churn'].value_counts(normalize=True).rename('percentage').mul(100).reset_index()

plt.figure(figsize=(12, 6))
churn_percentage['Churn'] = churn_percentage['Churn'].apply(lambda x: 0 if x == 'Ya' else 1)
bar_plot = sns.barplot(x=1, y='percentage', hue='Churn', data=churn_percentage, palette='viridis')

for p in bar_plot.patches:
    height = p.get_height()
    bar_plot.annotate(f'{height:.1f}%', (p.get_x() + p.get_width() / 2., height),
                      ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                      textcoords='offset points')

plt.title('Persentase Nilai Churn Berdasarkan Produk')
plt.xlabel('Nama Produk')
plt.ylabel('Persentase')
plt.legend(title='Churn')
plt.xticks(rotation=45)
plt.show()

### Analisa Bar Plot
Hampir semua produk memiliki angka invalid yang tinggi, dan juga hampir semua produk memiliki lebih banyak customer yang churn daripada tidak churn. Hal ini dapat dilihat pada setiap produk
* Product
  * lorem ipsum dolor

In [ ]:
churn_counts_per_product = dataset_filtered.groupby(1)['Churn'].value_counts().unstack(fill_value=0)

churn_percentage_per_product = churn_counts_per_product.div(churn_counts_per_product.sum(axis=1), axis=0).mul(100).stack().reset_index(name='percentage')

plt.figure(figsize=(12, 6))
churn_percentage_per_product['Churn'] = churn_percentage_per_product['Churn'].apply(lambda x: 0 if x == 'Ya' else 1)
bar_plot = sns.barplot(x=1, y='percentage', hue='Churn', data=churn_percentage_per_product, palette='viridis')

for p in bar_plot.patches:
    height = p.get_height()
    bar_plot.annotate(f'{height:.1f}%', (p.get_x() + p.get_width() / 2., height),
                      ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                      textcoords='offset points')

plt.title('Persentase Nilai Churn Berdasarkan Produk')
plt.xlabel('Nama Produk')
plt.ylabel('Persentase')
plt.legend(title='Churn')
plt.xticks(rotation=45)
plt.show()

In [ ]:
churn_counts = dataset_filtered.groupby(1)['Churn'].value_counts().rename('count').reset_index()

plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x=1, y='count', hue='Churn', data=churn_counts, palette='viridis')

for p in bar_plot.patches:
    height = p.get_height()
    bar_plot.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height),
                      ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                      textcoords='offset points')

plt.title('Jumlah Nilai Churn Berdasarkan Produk')
plt.xlabel('Nama Produk')
plt.ylabel('Jumlah')
plt.legend(title='Churn')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Harga produk
harga_produk = {
    "product": 000000
}

# Grup data berdasarkan produk dan status churn, hitung jumlahnya
churn_counts = dataset_filtered.groupby(1)['Churn'].value_counts().rename('count').reset_index()
print(churn_counts)

# Tambahkan kolom harga berdasarkan produk
churn_counts['Harga'] = churn_counts[1].map(harga_produk)

# Kalikan jumlah churn dengan harga produk
churn_counts['Total'] = churn_counts['count'] * churn_counts['Harga']

# Buat plot
plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x=1, y='Total', hue='Churn', data=churn_counts, palette='viridis')

# Tambahkan anotasi pada bar plot dengan format angka yang memiliki pemisah ribuan
for p in bar_plot.patches:
    height = p.get_height()
    bar_plot.annotate(f'{height:,.0f}', (p.get_x() + p.get_width() / 2., height),
                      ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                      textcoords='offset points')

# Tambahkan judul dan label
plt.title('Jumlah Nilai Churn Berdasarkan Produk (diibaratkan membeli kembali 1 box)')
plt.xlabel('Nama Produk')
plt.ylabel('Total Nilai (Rp)')
plt.legend(title='Churn')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Grup data berdasarkan produk dan status churn, hitung jumlahnya
churn_sum = dataset_filtered.groupby([1, 'Churn'])[9].sum().reset_index()
churn_sum['Churn'] = churn_sum['Churn'].apply(lambda x: 0 if x == 'Ya' else 1)

# Buat plot
plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x=1, y=9, hue='Churn', data=churn_sum, palette='viridis')

# Tambahkan anotasi pada bar plot dengan format angka yang memiliki pemisah ribuan
for p in bar_plot.patches:
    height = p.get_height()
    bar_plot.annotate(f'{height:,.0f}', (p.get_x() + p.get_width() / 2., height),
                      ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                      textcoords='offset points')

# Tambahkan judul dan label
plt.title('Jumlah Monetary Churn Berdasarkan Produk (jika membeli kembali dengan jumlah yang sama dengan sebelumnya)')
plt.xlabel('Nama Produk')
plt.ylabel('Jumlah Pengeluaran')
plt.legend(title='Churn')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
churn_sum = dataset_filtered.groupby([1, 'Churn'])[8].sum().reset_index()
churn_sum['Churn'] = churn_sum['Churn'].apply(lambda x: 0 if x == 'Ya' else 1)

plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x=1, y=8, hue='Churn', data=churn_sum, palette='viridis')

for p in bar_plot.patches:
    height = p.get_height()
    bar_plot.annotate(f'{int(height)}', (p.get_x() + p.get_width() / 2., height),
                      ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                      textcoords='offset points')

plt.title('Jumlah Box Berdasarkan Produk (Jika diibarkan dibeli dengan jumlah yang sama dengan sebelumnya)')
plt.xlabel('Nama Produk')
plt.ylabel('Jumlah')
plt.legend(title='Churn')
plt.xticks(rotation=45)
plt.show()

In [ ]:
valid_intervals = dataset_filtered[dataset_filtered['Interval'] != 'Invalid'].copy()
valid_intervals['Interval'] = pd.to_numeric(valid_intervals['Interval'], errors='coerce').fillna(0).astype(int)

statistics = valid_intervals.groupby(1)['Interval'].agg(['mean', 'median', 'min', 'max']).reset_index()

modus = valid_intervals.groupby(1)['Interval'].agg(lambda x: x.mode().tolist()).reset_index()
statistics = pd.merge(statistics, modus, on=1)
statistics.columns = ['Nama Produk', 'Mean', 'Median', 'Min', 'Max', 'Modus']

statistics

### Analisa Desc Stat
* Product
  * lorem ipsum dolor

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=1, y='Interval', data=valid_intervals)
plt.title('Box Plot Interval untuk Setiap Produk')
plt.xlabel('Nama Produk')
plt.ylabel('Interval')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

### Analisa Box Plot
Box plot ini menunjukkan bahwa rata-rata interval dan variabilitas produk [product] bla bla bla. Hal ini dapat disebabkan oleh beberapa faktor, seperti:
* lorem ipsum dolor

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=valid_intervals, x=1, y='Interval', hue=1, palette='viridis')
plt.title('Scatter Plot Interval untuk Setiap Produk')
plt.xlabel('Nama Produk')
plt.ylabel('Interval')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(title='Nama Produk')
plt.show()